# Chapter 8 - Example: Satellite Data 
###  Days with sea surface temperature above a threshold

In this chapter we exemplify the use of Sea Surface Temperature (SST) data in the cloud. 

This example analyzes a time series from an area of the ocean or a point. If an area, it averages SST values into a single value. Then it analyze the time series to assess when SST is above a given threshold. This could be used to study marine heatwaves, or use a SST threshold relevant to a marine species of interest.

<span style="font-size:larger;">You must have the Zarr package installed as well</span>

In [ ]:
import warnings 
warnings.simplefilter('ignore') 
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt 
import hvplot.pandas # this library helps to make interactive plots
import hvplot.xarray
import fsspec # these libraries help reading cloud data
import s3fs
import dask
from dask.distributed import performance_report, Client, progress

In [ ]:
# input parameters

# select either a range of lat/lon or a point. 
# If a point, set both entries to the same value
latr = [19, 20] # make sure lat1 < lat2 since no test is done below to simplify the code
lonr = [-158, -157] # lon1 < lon2, range -180:180. resolution daily 1km!

# time range. data range available: 2002-06-01 to 2020-01-20. [start with a short period]
dater = ['2012-01-01','2016-12-31'] # dates on the format 'YYYY-MM-DD' as string

***
## We are going to use the Multi-Scale Ultra High Resolution (MUR) Sea Surface Temperature (SST) data set
### This dataset is stored the Amazon (AWS) Cloud. For more info and links to the data detail and examples, see: https://registry.opendata.aws/mur/

This dataset is stored in `zarr` format, which is an optimized format for the large datasets and the cloud. It is not stored as one 'image' at a time or a gigantic netcdf file, but in 'chunks', so it is perfect for extracting time series.

First, we open the dataset and explore it, but we are not downloading anything yet.

In [ ]:
# first determine the file name using, in the format:
# the s3 bucket [mur-sst], and the region [us-west-2], and the folder if applicable [zarr-v1] 
file_location = 'https://mur-sst.s3.us-west-2.amazonaws.com/zarr-v1'

ds_sst = xr.open_zarr(file_location,consolidated=True) # open a zarr file using xarray
# it is similar to open_dataset but it only reads the metadata

ds_sst # we can treat it as a dataset!


## Now that we know what the file contains, we select our data (region and time), operate on it if needed (if a region, average), and download only the selected data 
It takes a while given the high resolution of the data. So, be patient.... and if you're only testing, might want to choose a small region and a short time period first. 

In [ ]:
# decide if a point or a region was given.
if (latr[0]==latr[1]) | (lonr[0]==lonr[1]): # if we give it only one point
    sst = ds_sst['analysed_sst'].sel(time = slice(dater[0],dater[1]),
                                            lat  = latr[0], 
                                            lon  = lonr[0]
                                           ).load()
else: # if we give it an area, it extract the area and average SST over the area and returns a time series of SST
    sst = ds_sst['analysed_sst'].sel(time = slice(dater[0],dater[1]),
                                            lat  = slice(latr[0], latr[1]), 
                                            lon  = slice(lonr[0], lonr[1])
                                           ).mean(dim={'lat','lon'}, skipna=True, keep_attrs=True).load() # skip 'not a number' (NaN) values and keep attributes

sst = sst-273.15 # transform units from Kelvin to  Celsius
sst.attrs['units']='deg C' # update units in metadata
sst.to_netcdf('data/sst_example.nc') # saving the data, incase we want to come back to analyze the same data, but don't want to acquire it again from the cloud.
sst # take a peak

***
### *Execute the next cell only if your reading the data from a file - either no access to cloud, or not want to keep reading from it. Skip otherwise. (No problem if you executed it by mistake).*

In [ ]:
sst = xr.open_dataset('data/sst_example.nc') 
sst.close()
sst = sst.analysed_sst # select only one variable
sst

***
## Let's plot the data using two different libraries.
#### - `matplotlib` that we already learn.
#### - `hovplot` is a more interactive library for web display. It provides you with the data details when you hover your cursor over the figure. Very nice for inspecting the data.

In [ ]:
# matplotlib method #
print('matplotlib') 
sst.plot() # this is all you need

# all the stuff here to make it look nice. 
plt.ylabel('SST ($^\circ$C)')
plt.xlabel('Year')
plt.title('Location: '+str(latr)+'$^\circ$N, '+str(lonr)+'$^\circ$W')
plt.grid(True, alpha=0.3)
plt.show()

# hovplot method #
print('hovplot')
df = pd.DataFrame(data=sst.data, index=sst.time.data,columns=['SST (C)'])
df.index.name = 'Date'
df.hvplot(grid=True)

***
## Now, let's analyze our data.
#### First, the basics: climatology and anomalies. Also plotting using `hovplot`.

In [ ]:
# Calculate the climatology
sst_climatology = sst.groupby('time.dayofyear').mean('time',keep_attrs=True,skipna=False) # Group by day, all years. skipna ignore missing (NaN) values 
sst_climstd = sst.groupby('time.dayofyear').std('time',keep_attrs=True,skipna=False) # Calculate standard deviation. Keep data attributes.

# creates a dataset with climatology and standard deviaton for easy plotting with hvplot
ds = xr.Dataset({'clim':sst_climatology,'+Std':sst_climatology+sst_climstd,'-Std':sst_climatology-sst_climstd}) # add standard deviation time series +/-
ds.hvplot(color=['k','grey','grey'], grid=True, title='SST Climatology') # plot the climatology (black, and the standard deviation in grey)

In [ ]:
# calculate the anomalies
sst_anomaly = sst.groupby('time.dayofyear')-sst_climatology 
sst_anomaly_monthly = sst_anomaly.resample(time='1MS', loffset='15D').mean(keep_attrs=True,skipna=False) # calculate monthly anomalies/smoothing

# make a plot 
plt.plot(sst_anomaly.time,sst_anomaly)
plt.plot(sst_anomaly_monthly.time,sst_anomaly_monthly, 'r')

plt.grid()
plt.ylabel('SSTa (C)')
plt.title('SST Anomalies')
plt.show()

***
## We analyze the data further by dividing it by a threshold.

- One way is to set a threshold that has some relevance.  For example, a thermal threshold for a marine species we are studying. 

- Another way is choosing the maximum value in the climatology (mean value + 1 standard deviation), which we can calculate or read by hovering our cursor over the climatology plot above.

### Once the threshold is choosen, we identify when SST is over that threshold, and count how many days that occurred each year

In [ ]:
# we define a function that take a threshold value, and analyze and plot our data
def SST_above(thr):
    
    fig, axs = plt.subplots(1,2,figsize=(16,4)) # creates a figure with two panels
    
    # first part - values above threshold - timeseries
    plt.subplot(1,2,1) # plot on the first panel (last number)
    plt.plot(sst.time,sst.data, lw=1)
    a=sst>=thr # test when data is equal or greater than the threshold. a is a logical array (True/False values)
    plt.plot(sst.time[a], sst.data[a],'.r', markersize=3) # plot only the values equal or above threshold
    # all stuff here to make it look good
    plt.ylabel('SST ($^\circ$C)')
    plt.xlabel('Year')
    plt.title('Location: '+str(latr)+'$^\circ$N, '+str(lonr)+'$^\circ$W')
    plt.grid(True, alpha=0.3)
    

    # second part - days per year above threshold
    plt.subplot(1,2,2) # plot on the second panel
    dts = sst[sst>=thr].time # select dates when SST is equal or greater than the threshold. note that this time is not a logical array, but the time values
    hot_days = dts.groupby('time.year').count() # agregate by year, by counting  
    plt.bar(hot_days.year, hot_days) # bar plot of days per year
    plt.xlim(int(dater[0][:4]), int(dater[1][:4])+1) # make it nice
    plt.ylabel('No. days above '+str(np.round(thr,1))+'C')
    plt.grid(True, alpha=0.3)
    plt.show() # display and finaiize this figure, so the next is not overwritten

## the actual analysis: two examples ##

### Maximum climatology threshold
thr = ds['+Std'].max() # setting threshold as maximum climatological value: mean + 1 standard deviation
print('Max climatological SST = ',np.round(thr,1),'C')
SST_above(thr) # Call function we defined

### A relevant threshold. 
# For example, for hawaii (the select region), 28C is a relevant threshold for coral bleaching (https://coralreefwatch.noaa.gov/product/5km/tutorial/crw08a_bleaching_threshold.php)
thr = 28
print('\n\nBiologically relevant SST = ',thr,'C')
SST_above(thr) # Call function

***
#### Now, a different analsys of anomalously warm SST days. 
## Marine heatwaves
Defined as any period with SST anomalies above the threshold determined by the 90th percentile value of a given period - in this case our data time period.

In [ ]:
# first, calculate the threshold: 90th percentile
thr = np.percentile(sst_anomaly, 90)

fig, axs = plt.subplots(3,1,figsize=(16,16)) # make a figure of 3 vertical panels

# same plot as in our function above, but this time we are plotting the anomalies.
plt.subplot(3,1,1) 
plt.plot(sst_anomaly.time,sst_anomaly.data, lw=1)
plt.axhline(y=0, c='k', zorder=0, alpha=0.5) # add a line to highlight the x axis 
a=sst_anomaly>=thr # select data above the threshold
plt.plot(sst_anomaly.time[a], sst_anomaly.data[a],'.r', markersize=3)
# all stuff here to make it look good
plt.ylabel('SST anomalies ($^\circ$C)')
plt.xlabel('Year')
plt.title('Location: '+str(latr)+'$^\circ$N, '+str(lonr)+'$^\circ$W')
plt.grid(True, alpha=0.3)

# Now plot on the original data (not anomalies)
plt.subplot(3,1,2) # second panel
plt.plot(sst.time,sst.data, lw=1)
plt.plot(sst.time[a], sst.data[a],'.r', markersize=3) # plot only the values equal or above threshold
# all stuff here to make it look good
plt.ylabel('SST ($^\circ$C)')
plt.xlabel('Year')
plt.title('Location: '+str(latr)+'$^\circ$N, '+str(lonr)+'$^\circ$W')
plt.grid(True, alpha=0.3)

# plot of marine heatwave days  per year
dts = sst_anomaly[sst_anomaly>=thr].time
mhw = dts.groupby('time.year').count()
plt.subplot(3,1,3) # third panel
plt.bar(mhw.year,mhw)
plt.ylabel('No. days SSTa > '+str(np.round(thr,1))+'C')
plt.grid(True, alpha=0.3)
plt.show()

mhw # print the numbers of days

## Resources

For the cloud and data in the cloud, see resources listed in Chapter 5.

### Resources specifically for this chapter:

- [MUR SST Data](https://registry.opendata.aws/mur/). SST data in the cloud, with references the official datta website, examples and other resources.

- [Pangeo OSM2020 Tutorial](https://github.com/pangeo-gallery/osm2020tutorial). This is a very good tutorial for ocean application and cloud computing. Plenty of examples. Many of the commands here are from this tutorial.

### About MHW

- [Marine heatwaves](http://www.marineheatwaves.org/all-about-mhws.html). A good place to begin to get info about the subject.

- [Marine heatwaves code](https://github.com/ecjoliver/marineHeatWaves). Marine heatwaves code from E. Oliver.

### If you want to learn more:

- [Methods for accessing a AWS bucket](https://docs.aws.amazon.com/AmazonS3/latest/userguide/access-bucket-intro.html). Bucket is the name of the cloud storage object. S3 stands for Amazon's Simple Storage Service.

- [hvplot site](https://hvplot.holoviz.org/index.html). Plotting tool used here.

- [zarr](https://zarr.readthedocs.io/en/stable/). Learn more about this big data storage format.